**Import Library**

In [ ]:
!pip install tensorflowjs

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflowjs as tfjs

**Load Dataset**

In [ ]:
df = pd.read_csv("/content/Dataset_Stunting.csv")
df = df.dropna()

**Encode Target**

In [ ]:
label_encoder = LabelEncoder()
df['status_gizi_who_encoded'] = label_encoder.fit_transform(df['status_gizi_who'])

**Fitur dan label**

In [ ]:
X = df.drop(['status_gizi_who', 'status_gizi_who_encoded'], axis=1)
y = df['status_gizi_who_encoded']

**Encode kolom kategorikal**

In [ ]:
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

**Normalisasi**

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

**Model Neural Network**

In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Train**

In [ ]:
history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.2)

Epoch 1/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.8403 - loss: 0.3786 - val_accuracy: 0.9049 - val_loss: 0.2120
Epoch 2/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8960 - loss: 0.2326 - val_accuracy: 0.9109 - val_loss: 0.1923
Epoch 3/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9030 - loss: 0.2125 - val_accuracy: 0.9110 - val_loss: 0.1903
Epoch 4/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9042 - loss: 0.2072 - val_accuracy: 0.9152 - val_loss: 0.1851
Epoch 5/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9060 - loss: 0.2033 - val_accuracy: 0.9162 - val_loss: 0.1776
Epoch 6/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9049 - loss: 0.2001 - val_accuracy: 0.9177 - val_loss: 0.1758
Epoch 7/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9076 - loss: 0.1981 - val_accuracy: 0.9174 - val_loss: 0.1753
Epoch 8/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9074 - loss: 0.19

**Evaluasi**

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nAkurasi pada data uji: {accuracy:.2f}")

756/756 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9540 - loss: 0.1045
756/756 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9540 - loss: 0.1045

Akurasi pada data uji: 0.95

Akurasi pada data uji: 0.95


**Prediksi**

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

756/756 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


**Hasil**

In [ ]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(
    y_test, y_pred, target_names=label_encoder.classes_
))


Confusion Matrix:
 [[17369     6   221]
 [    0  3944   168]
 [  319   392  1756]]

Classification Report:
                   precision    recall  f1-score   support

          normal       0.98      0.99      0.98     17596
severely stunted       0.91      0.96      0.93      4112
         stunted       0.82      0.71      0.76      2467

        accuracy                           0.95     24175
       macro avg       0.90      0.89      0.89     24175
    weighted avg       0.95      0.95      0.95     24175


Confusion Matrix:
 [[17369     6   221]
 [    0  3944   168]
 [  319   392  1756]]

Classification Report:
                   precision    recall  f1-score   support

          normal       0.98      0.99      0.98     17596
severely stunted       0.91      0.96      0.93      4112
         stunted       0.82      0.71      0.76      2467

        accuracy                           0.95     24175
       macro avg       0.90      0.89      0.89     24175
    weighted avg       

**SIMPAN MODEL DAN ALAT PRA-PROSES**

In [ ]:
model.save("model_stunting_nn.h5")
joblib.dump(scaler, "scaler_stunting.pkl")
joblib.dump(label_encoder, "label_encoder_stunting.pkl")

['label_encoder_stunting.pkl']

['label_encoder_stunting.pkl']

In [ ]:
tfjs.converters.save_keras_model(model, "tfjs_model")
print("\n✅ Model berhasil dikonversi ke TensorFlow.js di folder 'tfjs_model/'")

failed to lookup keras version from the file,
    this is likely a weight only file

✅ Model berhasil dikonversi ke TensorFlow.js di folder 'tfjs_model/'
failed to lookup keras version from the file,
    this is likely a weight only file

✅ Model berhasil dikonversi ke TensorFlow.js di folder 'tfjs_model/'
